<a href="https://colab.research.google.com/github/dushyanthbommana/CE888/blob/main/LAB3/bank_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab3/bank-additional-full.csv", delimiter = ";")


In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum().sum()

In [ ]:
df.columns.values

drop duration column

In [ ]:
df.drop('duration', inplace=True, axis=1)

label encode only the target columns which is y 

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['y']= label_encoder.fit_transform(df['y'])
df['y'].unique()

In [ ]:
features = ["age", "job", "marital", "education", "default", "housing", "loan",
       "contact", "month", "day_of_week", "campaign", "pdays",
       "previous", "poutcome", "emp.var.rate", "cons.price.idx",
       "cons.conf.idx", "euribor3m", "nr.employed"]


df[features].head()

In [ ]:
df.month.unique()

In [ ]:
df['y'].head()

In [ ]:
df_dummies_features = pd.get_dummies(df[features])

In [ ]:
df_dummies_features.head()

In [ ]:
df_dummies_features.info()

In [ ]:
df_dummies_features.columns.values

In [ ]:
df_dummies_features.dropna(axis=0,inplace=True)

In [ ]:
len(df_dummies_features)

In [ ]:
df_dummies_features.isnull().sum()

In [ ]:
df['y'].hist()

calculating imbalance in the target variable

In [ ]:
class_count_0, class_count_1 = df['y'].value_counts()
class_0 = df[df['y'] == 0]
class_1 = df[df['y'] == 1]
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

In [ ]:
import imblearn
import collections
from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
X=df_dummies_features
Y=df['y']

MODELLING THE DATA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=50, stratify=Y)

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(max_depth=6,n_estimators=750,random_state=60)
model.fit(X_train,y_train)
model_predict=model.predict(X_test)

In [ ]:
import pandas as pd
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10)
n_scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

In [ ]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, model_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, model_predict))
print('\n')

In [ ]:
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Training dummy classifier

In [ ]:
dumm_clf = DummyClassifier(strategy='stratified')
dumm_clf.fit(X, Y)
print("Dummy ACC: %.2f" % acc(Y, dumm_clf.predict(X)))

grid search cv on random forest classifier

In [ ]:
param_grid = { 
    'n_estimators': [100, 200,300],
    'max_depth' : [5,7,9,11],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']
}


In [ ]:
from sklearn.model_selection import GridSearchCV
model1=RandomForestClassifier()
CV_rfc = GridSearchCV(estimator=model1, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

In [ ]:
CV_rfc.best_params_

In [ ]:
from sklearn.metrics import accuracy_score
pred=CV_rfc.predict(X_test)
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

In [ ]:
y_error = ((y_test - pred)**2).mean()
y_error